## Tool to get package details
This tool will be used later in langchain to get the details of packages on system which are vulnerable along with thier versions. This data can be fed to AI so it can further analyse on the details.

*Note: This is first iteration code we are no longer going to use this in final project*

In [ ]:
import subprocess
from collections import namedtuple, defaultdict
from typing import Dict, List, Optional
from pydantic import BaseModel, Field

class PackageDetails(BaseModel):
    version: Optional[str] = Field(default=None, description="Installed package version")
    vulnerabilities: List[str] = Field(default_factory=list, description="List of vulnerability IDs")

def get_package_versions(package_names: List[str]) -> Dict[str, Optional[str]]:
    """ Get installed package versions for multiple packages using single dpkg call

    Args:
        - package_names (List[str]): List of package names

    Returns:
        - Dict[str, Optional[str]]: Package name to version mapping
    """
    if not package_names:
        return {}
    
    versions = {}
    try:
        # Use dpkg to query all packages at once
        cmd = ["dpkg-query", "-W", "-f=${Package}:${Version}\n"] + package_names
        out = subprocess.run(cmd, capture_output=True, text=True, check=False).stdout
        
        for line in out.splitlines():
            if ':' in line:
                package, version = line.split(':', 1)
                versions[package] = version
                
        # Set None for packages that weren't found
        for package in package_names:
            if package not in versions:
                versions[package] = None
                
    except subprocess.CalledProcessError:
        # If batch query fails, return None for all packages
        versions = {package: None for package in package_names}
    
    return versions

def get_package_details() -> Dict[str, PackageDetails]:
    """ Get installed package details including vulnerability

    This function uses debsecan and dpkg to find installed version as well as active
    vulnerabilities in the package.

    Returns:
        - Dict[str, PackageDetails]: Package as key and PackageDetails as value
    """
    try:
        out = subprocess.run(["debsecan"], capture_output=True, text=True, check=True).stdout
        
        # First pass: collect all unique packages and their vulnerabilities
        package_vulnerabilities = defaultdict(list)
        for line in out.splitlines():
            parts = line.split()
            if len(parts) >= 2:
                vulnerability_id = parts[0]
                package = parts[1]
                package_vulnerabilities[package].append(vulnerability_id)
        
        # Get versions for all unique packages in a single batch call
        unique_packages = list(package_vulnerabilities.keys())
        package_versions = get_package_versions(unique_packages)
        
        # Build final result
        package_details = {}
        for package, vulnerabilities in package_vulnerabilities.items():
            package_details[package] = PackageDetails(
                version=package_versions.get(package),
                vulnerabilities=vulnerabilities
            )
        
        return package_details
        
    except subprocess.CalledProcessError as e:
        print(f"Error running debsecan: {e}")
        return {}

## System Scanner Tool
This tool scans the local system for vulnerabilities using `debsecan` and enriches CVE identifiers with EPSS data from the First.org API.
It registers a LangChain `@tool` named `scan_system_vulnerabilities(top=5)` that returns a human-readable string reporting how many vulnerabilities were found and lists the top `top` vulnerabilities sorted by EPSS score.
The `Vulnerability` pydantic model includes `cve`, `epss`, `percentile`, and an optional `date` field.
The function handles `debsecan` and network errors and returns an error message on failure. Note: the default `top` is 5 (not 10).

In [ ]:
import requests
import gzip
import csv

def download_epss_csv(dest_path="epss_scores-current.csv.gz") -> bool:
    """Downloads the latest EPSS CSV file.

    Args:
        dest_path (str): Path to save the downloaded file.
    Returns:
        bool: True if download succeeded, False otherwise.
    """
    url = "https://epss.empiricalsecurity.com/epss_scores-current.csv.gz"
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
        with open(dest_path, 'wb') as f:
            f.write(response.content)
        return True
    except Exception as e:
        print(f"Update failed: {e}. Falling back to existing file.")
        return False
    
def load_epss_map(file_path="epss_scores-current.csv.gz"):
    """Loads EPSS data into a dictionary indexed by CVE ID."""
    epss_map = {}
    with gzip.open(file_path, mode='rt') as f:
        # Skip FIRST.org header metadata lines
        reader = csv.DictReader(filter(lambda row: not row.startswith('#'), f))
        for row in reader:
            # Map CVE ID to its (epss, percentile) tuple
            epss_map[row['cve']] = (row['epss'], row['percentile'])
    return epss_map

download_epss_csv()

In [ ]:
import subprocess
import requests
from typing import Optional, Set
from pydantic import BaseModel, Field
from functools import lru_cache

from langchain.tools import tool


class Vulnerability(BaseModel):
    cve: str = Field(description="Vulnerability identifier, e.g., 'CVE-2023-4863'")
    epss: float = Field(description="Probability of exploitation (0.0-1.0)")
    percentile: float = Field(description="Percentile rank of this CVE rank")
    date: Optional[str] = Field(None, description="Date of the EPSS data retrieval")

@lru_cache(maxsize=1024)
def batch_cves_for_url_limit(cves: tuple, max_url_length: int = 2000, base_url: str = "https://api.first.org/data/v1/epss") -> list:
    """Split CVEs into batches that keep URL length under the specified limit.
    
    Args:
        cves: List of CVE identifiers
        max_url_length: Maximum allowed URL length (default: 2000)
        base_url: Base URL for the API endpoint
    
    Returns:
        List of CVE batches, where each batch keeps URL under the limit
    """
    if not cves:
        return []
    
    batches = []
    current_batch = []
    
    # Account for base URL and query parameter structure
    base_length = len(base_url) + len("?cve=")
    
    for cve in cves:
        # Calculate what the URL length would be if we add this CVE
        # Format: ?cve=CVE-2023-1&cve=CVE-2023-2 etc.
        if current_batch:
            test_length = base_length + len("&cve=".join(current_batch + [cve]))
        else:
            test_length = base_length + len(cve)
        
        # If adding this CVE would exceed the limit, start a new batch
        if test_length > max_url_length and current_batch:
            batches.append(current_batch)
            current_batch = [cve]
        else:
            current_batch.append(cve)
    
    # Add the last batch if it has items
    if current_batch:
        batches.append(current_batch)
    
    return batches

@lru_cache(maxsize=1024)
def fetch_epss_data(cve_list: tuple) -> List[Vulnerability]:
    """Fetch EPSS data for a list of CVEs.
    
    Args:
        cve_list (List[str]): List of CVE identifiers

    Returns:
        List[Vulnerability]: List of Vulnerability data
    """
    vulnerabilities = []
    
    # Batch CVEs to keep URL length manageable
    cve_batches = batch_cves_for_url_limit(tuple(cve_list), max_url_length=2000)
    
    for batch in cve_batches:
        # Build query parameters for GET request
        params = {"cve": batch}
        response = requests.get("https://api.first.org/data/v1/epss", params=params)
        response.raise_for_status()
        data = response.json()
        
        # Parse and add to vulnerabilities list
        batch_vulnerabilities = [Vulnerability(**item) for item in data.get("data", [])]
        vulnerabilities.extend(batch_vulnerabilities)
    
    return vulnerabilities

@tool
def scan_system_vulnerabilities(top: int = 5) -> Dict[str, List[Vulnerability]]:
    """Scan system for vulnerabilities and return top N sorted by EPSS score.
    
    Args:
        top (int): Number of top vulnerabilities to return (default: 5)
    
    Returns:
        Dict[str, List[Vulnerability]]: Dictionary with all vulnerabilities and top N vulnerabilities
    """
    try:
        out = subprocess.run(["debsecan"], capture_output=True, text=True, check=True).stdout
        vulnerabilities = {cve.split()[0] for cve in out.splitlines()}
        
        if not vulnerabilities:
            return {"all_vulnerabilities": [], "top_vulnerabilities": []}
        
        # Batch CVEs to keep URL length manageable
        cve_list = list(vulnerabilities)
        cve_batches = batch_cves_for_url_limit(tuple(cve_list), max_url_length=2000)
        
        all_vulnerability_data = []
        
        # Process each batch
        for batch in cve_batches:
            batch_vulnerabilities = fetch_epss_data(tuple(batch))
            all_vulnerability_data.extend(batch_vulnerabilities)
        
        # Sort by EPSS score (highest first)
        sorted_vulnerabilities = sorted(all_vulnerability_data, key=lambda v: v.epss, reverse=True)
        
        # Format the top N vulnerabilities
        top_vulns = sorted_vulnerabilities[:top]
        
        return {
  #          "all_vulnerabilities": sorted_vulnerabilities,
           "top_vulnerabilities": top_vulns
        }
        
    except requests.RequestException as e:
        print(f"Error fetching the EPSS data for vulnerabilities: {e}")
        return {"all_vulnerabilities": [], "top_vulnerabilities": []}
    except subprocess.CalledProcessError as e:
        print(f"Error running debsecan: {e}")
        return {"all_vulnerabilities": [], "top_vulnerabilities": []}

In [ ]:
print("Top 5 vulnerabilities:", scan_system_vulnerabilities(5))
#print("Top 10 vulnerabilities:", scan_system_vulnerabilities(10))

Function to return Debian version in a format liked by osv.dev. Note that sid/unstable is not recognized and probably falls under Debian:14.

In [ ]:
import re
def get_debian_version() -> Optional[str]:
    """Detect the current Debian version from the system"""
    try:
        # Try to read from /etc/debian_version
        with open('/etc/debian_version', 'r') as f:
            version = f.read().strip()
            # Extract major version number (e.g., "12.7" -> "12")
            match = re.match(r'(\d+)', version)
            if match:
                return match.group(1)
            match = re.match(r'.*\/sid', version)
            if match:
                return "14"
    except (FileNotFoundError, IOError):
        pass
    
    return None

## Getting CVE data using osv.dev
We use osv.dev which is effort by Google to have all vulnerability related data in single place, irrespective of OS and CVE providers and in a consumable format. We use pydantic to define classes for parsing required information from the osv.dev api into proper python object. OSV schema is defined at https://ossf.github.io/osv-schema/ and a sample Debian CVE data from OSV is https://api.osv.dev/v1/vulns/DEBIAN-CVE-2024-7883

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Optional, Dict, Any, Union
import subprocess
import re

class Severity(BaseModel):
    type: str = Field(description="Severity type, e.g., 'CVSSv3'")
    score: str = Field(description="Severity score, e.g., 'CVSS:3.1/AV:N/AC:H/PR:N/UI:N/S:U/C:L/I:N/A:N'")

    def __str__(self) -> str:
        return f"{self.score}"
    
class Package(BaseModel):
    name: str = Field(description="Name of the package")
    ecosystem: str = Field(description="Ecosystem of the package, e.g., 'Debian:12'")
    purl: Optional[str] = Field(None, description="Package URL")

    def __str__(self) -> str:
        return f"{self.name}"

class Event(BaseModel):
    """Represents an event in a version range (introduced, fixed, etc.)"""
    introduced: Optional[str] = Field(None, description="Version where vulnerability was introduced")
    fixed: Optional[str] = Field(None, description="Version where vulnerability was fixed")
    last_affected: Optional[str] = Field(None, description="Last version affected by vulnerability")
    limit: Optional[str] = Field(None, description="Limit version for the range")
    
    def is_fixed_event(self) -> bool:
        """Check if this event represents a fix"""
        return self.fixed is not None
    
    def get_fixed_version(self) -> Optional[str]:
        """Get the fixed version if this is a fix event"""
        return self.fixed
    
    def __str__(self) -> str:
        if self.introduced:
            return f"Introduced in {self.introduced}"
        if self.fixed:
            return f"Fixed in {self.fixed}"
        return "Not fixed"

class Range(BaseModel):
    """Represents a version range with events"""
    type: str = Field(description="Range type, e.g., 'ECOSYSTEM', 'SEMVER'")
    events: List[Event] = Field(description="List of events in this range")
    repo: Optional[str] = Field(None, description="Repository URL for this range")
    
    def has_fixed_event(self) -> bool:
        """Check if this range contains any fixed events"""
        return any(event.is_fixed_event() for event in self.events)
    
    def get_fixed_versions(self) -> List[str]:
        """Get all fixed versions in this range"""
        return [event.get_fixed_version() for event in self.events if event.is_fixed_event()]
    
    def get_latest_fixed_version(self) -> Optional[str]:
        """Get the latest fixed version in this range"""
        fixed_versions = self.get_fixed_versions()
        return fixed_versions[-1] if fixed_versions else None

class Affected(BaseModel):
    package: Package = Field(description="Affected package details")
    versions: Optional[List[str]] = Field(None, description="List of affected versions")
    ranges: Optional[List[Range]] = Field(None, description="Affected version ranges")
    ecosystem_specific: Optional[Dict[str, Any]] = Field(None, description="Ecosystem-specific data")
    database_specific: Optional[Dict[str, Any]] = Field(None, description="Database-specific data")
    
    def has_any_fix(self) -> bool:
        """Check if any range in this affected package has a fix"""
        if not self.ranges:
            return False
        return any(range_item.has_fixed_event() for range_item in self.ranges)
    
    def get_all_fixed_versions(self) -> List[str]:
        """Get all fixed versions across all ranges"""
        if not self.ranges:
            return []
        
        fixed_versions = []
        for range_item in self.ranges:
            fixed_versions.extend(range_item.get_fixed_versions())
        return fixed_versions

class Reference(BaseModel):
    type: str = Field(description="Reference type, e.g., 'ADVISORY'")
    url: str = Field(description="Reference URL")

    def __str__(self) -> str:
        return f"{self.type}: {self.url}"


class DebianCVE(BaseModel):
    id: str = Field(description="CVE identifier, e.g., 'CVE-2023-4863'")
    details: str = Field(description="Detailed description of the CVE")
    modified: Optional[str] = Field(None, description="Modification timestamp")
    published: Optional[str] = Field(None, description="Publication timestamp")
    upstream: Optional[List[str]] = Field(None, description="Upstream references")
    references: Optional[List[Reference]] = Field(None, description="List of references")
    severity: Optional[List[Severity]] = Field(None, description="List of severity assessments")
    affected: List[Affected] = Field(description="List of affected packages and versions")
    
    def has_fixes_available(self) -> bool:
        """Check if any affected package has fixes available"""
        return any(affected_pkg.has_any_fix() for affected_pkg in self.affected)
    
    def get_packages_with_fixes(self) -> List[Affected]:
        """Get all affected packages that have fixes available"""
        return [affected_pkg for affected_pkg in self.affected if affected_pkg.has_any_fix()]
    
    def get_packages_without_fixes(self) -> List[Affected]:
        """Get all affected packages that don't have fixes available"""
        return [affected_pkg for affected_pkg in self.affected if not affected_pkg.has_any_fix()]
    
    def filter_for_system_version(self, version: str) -> 'DebianCVE':
        """Filter for a specific Debian version (e.g., '12', '13')"""
        target_ecosystem = f"Debian:{version}"
        filtered_affected = [
            affected for affected in self.affected 
            if affected.package.ecosystem == target_ecosystem
        ]
        
        # Create a new instance with filtered data
        return DebianCVE(
            id=self.id,
            details=self.details,
            modified=self.modified,
            published=self.published,
            upstream=self.upstream,
            references=self.references,
            severity=self.severity,
            affected=filtered_affected
        )
    
    def filter_for_current_system(self) -> 'DebianCVE':
        """Return a new DebianCVE instance filtered for the current Debian version"""
        current_version = get_debian_version()
        if not current_version:
            # If we can't detect the version, return the original
            return self
        
        return self.filter_for_system_version(current_version)
    
    def get_available_ecosystems(self) -> List[str]:
        """Get list of all ecosystems in the affected packages"""
        return list(set(affected.package.ecosystem for affected in self.affected))
    
    def _format_references(self) -> str:
        """Format references section"""
        if not self.references:
            return ""
        
        lines = ["References:\n"]
        for ref in self.references:
            lines.append(f"  - {ref}\n")
        return "".join(lines)
    
    def _format_severity(self) -> str:
        """Format severity section"""
        if not self.severity:
            return ""
        
        lines = ["Severity:\n"]
        for sev in self.severity:
            lines.append(f"  - {sev}\n")
        return "".join(lines)
    
    def _format_versions(self, versions: List[str]) -> str:
        """Format versions section"""
        if not versions:
            return ""
        
        lines = ["  Versions:\n"]
        for version in versions:
            lines.append(f"    - {version}\n")
        return "".join(lines)
    
    def _format_fix_status(self, affected: Affected) -> str:
        """Format fix status for an affected package"""
        if not affected.ranges:
            return ""
        
        fixed_events = [
            event for range_item in affected.ranges 
            for event in range_item.events 
            if event.is_fixed_event()
        ]
        
        if fixed_events:
            lines = []
            for event in fixed_events:
                if not event.introduced:
                    lines.append(f"      * Status: {event}\n")
            return "".join(lines)
        else:
            return "  No fixes available.\n"
    
    def __str__(self) -> str:
        lines = [f"CVE: {self.id}\nDescription: {self.details}\n"]
        
        lines.append(self._format_references())
        lines.append(self._format_severity())
        
        for affected in self.affected:
            lines.append(f"Affected Package: {affected.package}\n")
            lines.append(self._format_versions(affected.versions))
            lines.append(self._format_fix_status(affected))
        
        return "".join(lines)

    def to_llm_summary(self, epss: str, percentile: str) -> str:
        """Generates a concise summary optimized for LLM reasoning."""
        summary = [
            f"CVE_ID: {self.id}",
            f"CRITICALITY: {self.severity[0].score if self.severity else 'Unknown'}",
            f"DETAILS: {self.details[:300]}..." # Keep descriptions short
        ]
        
        if not self.affected:
            summary.append("SYSTEM_IMPACT: No packages affected on this Debian version.")
        else:
            for affected_pkg in self.affected:
                pkg_name = affected_pkg.package.name
                fixed_versions = affected_pkg.get_all_fixed_versions()
                
                status = "FIX_AVAILABLE" if fixed_versions else "AWAITING_FIX"
                fix_detail = f"Fixed in: {', '.join(fixed_versions)}" if fixed_versions else "No fix version assigned yet."
                
                summary.append(f"PACKAGE: {pkg_name} | STATUS: {status} | {fix_detail}")

        summary.append(f"EPSS_SCORE: {epss} | PERCENTILE: {percentile}")
        
        return "\n".join(summary)

In [ ]:
import requests
from langchain.tools import tool

#@tool
def research_vulnerability(cve_id: str, epss: str, percentile: str) -> str:
    """Fetch vulnerability details from the Debian CVE API
    
    Args:
        cve_id (str): CVE identifier, e.g., 'CVE-2023-4863'
        epss (str): EPSS score as string
        percentile (str): Percentile rank as string

    Returns:
        str: Human-readable summary of the vulnerability
    """
    try:
        url = f"https://api.osv.dev/v1/vulns/DEBIAN-{cve_id}"
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        
        debian_cve = DebianCVE(**data)
        debian_cve_filtered = debian_cve.filter_for_current_system()
        
        return debian_cve_filtered.to_llm_summary(epss, percentile)
        
    except requests.RequestException as e:
        return f"Error fetching CVE data for {cve_id}: {e}"

In [ ]:


print(research_vulnerability("CVE-2026-21945", "0.0004", "0.11914"))

In [ ]:
import subprocess
from langchain.tools import tool
from typing import List


epss_lookup = load_epss_map()


@tool
def scan_system_vulnerabilities_llama(top: int = 5) -> List[str]:
    """Scan system for vulnerabilities and return top N sorted by EPSS score.
    
    Args:
        top (int): Number of top vulnerabilities to return (default: 5)

    Returns:
        List[str]: List of CVE IDs of the top vulnerabilities
    """

    try:
        out = subprocess.run(["debsecan"], capture_output=True, text=True, check=True).stdout
        vulnerabilities = {cve.split()[0] for cve in out.splitlines()}
        
        if not vulnerabilities:
            return []
        
        # Sort vulnerabilities by EPSS score using the preloaded epss_lookup
        sorted_vulnerabilities = sorted(
            vulnerabilities,
            key=lambda cve: float(epss_lookup.get(cve, (0, 0))[0]),
            reverse=True
        )
        
        return sorted_vulnerabilities[:top]

    except subprocess.CalledProcessError as e:
        print(f"Error running debsecan: {e}")
        return []

@tool
def research_vulnerability_llama(cve_id: str) -> str:
    """
    MANDATORY: Research a SINGLE vulnerability. 
    If you have 10 CVE IDs, you must call this tool 10 times.
    Input MUST be a string (e.g., 'CVE-2023-1234'). 
    Do NOT pass lists or multiple IDs.
    """
    epss, percentile = epss_lookup.get(cve_id, ("0", "0"))
    try:
        return research_vulnerability(cve_id, epss, percentile)
    except Exception as e:
        return f"TOOL ERROR: Error researching vulnerability {cve_id}: {e}"

In [ ]:
#prompts
simple_prompt = "You are Debian Security Expert. Analyse system for vulnerabilities and prioritise fixes."

# fine tuned prompt
system_prompt = """You are a Debian Security Expert.
CRITICAL PROCESS:
 1. First, use 'scan_system_vulnerabilities' to find the most pressing issues.
 2. For EACH of the top CVEs found in the scan, you MUST use 'research_vulnerability' to get the actual fix status and technical details.
 3. DO NOT guess fix versions or dates. If you haven't called the research tool, you do not have the data.
 4. Only after researching all top CVEs should you provide your final prioritized report."""

strict_workflow_prompt = """You are a Debian Security Expert. 
Your goal is to provide a fully researched report.
 STRICT WORKFLOW:
  1. Call 'scan_system_vulnerabilities'.
  2. For EVERY CVE returned in the top results, you MUST IMMEDIATELY call 'research_vulnerability'.
  3. DO NOT ask the user for permission. Researching is your default required behavior.
  4. Only once you have the fix details from the researcher tool should you give the final answer."""

few_shot_prompt = """You are a Debian Security Expert. You must be autonomous and thorough.

STRICT WORKFLOW:
1. Scan the system.
2. For EVERY CVE found, immediately call 'research_vulnerability'.
3. Do not report findings until you have the 'research_vulnerability' results.

---
EXAMPLE OF CORRECT BEHAVIOR:
User: Scan my system and tell me the top 2 vulnerabilities.
Thought: I need to scan the system first.
Action: scan_system_vulnerabilities
Action Input: {"top": 2}
Observation: {"top_vulnerabilities": [{"cve": "CVE-2024-1234", "epss": 0.05}, {"cve": "CVE-2024-5678", "epss": 0.01}]}

Thought: I have the scan results. I must now research CVE-2024-1234 to get fix details.
Action: research_vulnerability
Action Input: {"cve_id": "CVE-2024-1234", "epss": "0.05", "percentile": "0.9"}
... (repeat for other CVEs)
Thought: Now I have all the data to provide a final report.
Final Answer: [Detailed Report]
---
"""

threatening_prompt = """You are a Debian Security Expert.
# STRICT RULE: The 'scan_system_vulnerabilities' tool ONLY provides IDs. 
# You are FORBIDDEN from reporting any details about a CVE until you have called 'research_vulnerability' for that ID.
# If you summarize the scan without researching, you have failed your mission."""

strict_workflow_prompt = """You are a Debian Security Expert.
1. Run 'scan_system_vulnerabilities_llama' to get the IDs.
2. For EVERY ID returned, you MUST call 'research_vulnerability_llama'.
3. DO NOT EXPLAIN what you are doing. DO NOT narrate. 
4. DO NOT provide a final report until you have the output from the research tool for every CVE.
If you talk before researching, the system will fail."""


strict_logic_prompt = """You are a Debian Security Logic Engine. 
You must follow this EXACT sequence without any conversational filler:
1. Run 'scan_system_vulnerabilities_llama'.
2. For EVERY CVE ID in the output, call 'research_vulnerability_llama' immediately.
3. DO NOT say "I will now research" or "Let me check."
4. ONLY provide a final summary after ALL research tool calls are finished.
If you output JSON as text instead of a tool call, you have failed."""

strict_logic_prompt = """You are a Debian Security Logic Engine. 
You are NOT allowed to speak to the user until ALL research is done.
STRICT WORKFLOW:
1. Call 'scan_system_vulnerabilities_llama'.
2. For EVERY CVE ID in the output, call 'research_vulnerability_llama' IMMEDIATELY.
3. DO NOT say "I will now research" or "Let me check."
4. DO NOT provide any text until you have the final research results.
Your response MUST be a tool call, not text."""

In [ ]:
from langchain_ollama.chat_models import ChatOllama
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain_core.prompts import PromptTemplate

llm = ChatOpenAI(
    model="llama3.1",
    api_key="ollama",
    base_url="http://localhost:11434/v1",
    temperature=0,
    stop=["I will", "Now I will", "Step", "Plan:"]
)

# Define your tool list (using the tools you've already built)
tools = [scan_system_vulnerabilities_llama, research_vulnerability_llama]

# Construct the ReAct agent
agent = create_agent(model=llm, 
                     tools=tools,
                     system_prompt=strict_logic_prompt,)                   


# The input format remains the same as your .invoke() call
inputs = {
    "messages": [
        {"role": "user", "content": "Scan my system and tell me the top 3 vulnerabilities."}
    ]
}

# Iterate through the stream
for chunk in agent.stream(inputs, stream_mode="updates"):
    for step_name, data in chunk.items():
        print(f"\n[Step: {step_name}]")
        
        # Pull the last message from this step's update
        last_msg = data["messages"][-1]
        
        # Use pretty_print() for a clean display of each step
        last_msg.pretty_print()
